# Imports

In [1]:
"""
LLM to synthesize job descriptions from freeform text.
Uses "Salesforce/xgen-7b-8k-inst" LLM from Hugging Face.
"""

# Load packages
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline #AutoConfig, 
from accelerate import Accelerator, load_checkpoint_and_dispatch

from huggingface_hub import login
# import streamlit as st
# from pathlib import Path

In [2]:
torch.cuda.is_available()

True

In [3]:
import requests
url = "https://huggingface.co/"
requests.get(url, verify=True)

<Response [200]>

In [4]:
login()

In [4]:
# Load user-defined modules
from config import AppConfig

# Instantiate app configuration
config = AppConfig()
SHARDS = config.SHARDS
SALESFORCE = config.SALESFORCE
CACHE = config.CACHE
ACCESS_KEY = config.API_TOKEN
# HF_HOME = config.HF_HOME
# HF_DATASETS_CACHE = config.HF_DATASETS_CACHE
# TRANSFORMERS_CACHE = config.TRANSFORMERS_CACHE

In [5]:
import requests

API_URL = "https://api-inference.huggingface.co/models/Salesforce/xgen-7b-8k-inst"
headers = {"Authorization": f"Bearer {ACCESS_KEY}"}

In [6]:
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

In [7]:
output = query({
	"inputs": "Can you please let us know more details about your position as a data scientist?",
})

In [8]:
print(output)

{'error': 'The model Salesforce/xgen-7b-8k-inst is too large to be loaded automatically (27GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).'}


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "Salesforce/xgen-7b-8k-inst", 
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

d:\__python__\_projects_\_jibr_jobr\metro\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\schel\.cache\huggingface\hub\models--Salesforce--xgen-7b-8k-inst. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenization_xgen.py:   0%|          | 0.00/8.85k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Salesforce/xgen-7b-8k-inst:
- tokenization_xgen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [8]:
from transformers import BitsAndBytesConfig

config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "Salesforce/xgen-7b-8k-inst", 
    torch_dtype=torch.bfloat16,
    quantization_config=config,
    ##  DEPRECATED  load_in_8bit=True,
)


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('Salesforce/xgen-7b-8k-inst')}
The following directories listed in your path were found to be non-existent: {WindowsPath('content/model')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('//matplotlib_inline.backend_inline')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: 7.5.
CUDA SETUP: To manually override the PyTorc

d:\__python__\_projects_\_jibr_jobr\metro\Lib\site-packages\bitsandbytes\cuda_setup\main.py:167: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
# Load user-defined modules
from config import AppConfig

# Instantiate app configuration
config = AppConfig()
SHARDS = config.SHARDS

In [ ]:
save_directory = SHARDS
accelerator = Accelerator()

In [ ]:
accelerator.save_model(model=model, 
                       save_directory=save_directory,
                       max_shard_size="2GB",
)

In [ ]:
model = load_checkpoint_and_dispatch(model, 
                                     checkpoint=SHARDS,
                                     device_map="auto",
                                     no_split_module_classes=['Block'],
)

# Test inference
## Let's see how i sharded, hmm?

In [ ]:
raw_inputs = "Can you tell me what a data scientist does?"

In [ ]:
inputs = tokenizer(raw_inputs,
                   padding=True,
                   truncation=True,
                   return_tensors="pt",
                   max_length=1024,
                   )

In [ ]:
outputs = model.generate(**inputs, 
                         max_new_tokens=1024,
                         return_dict_in_generate=True, 
                         output_scores=True,
                         )

In [ ]:
print(outputs)

In [3]:
from transformers import pipeline

In [ ]:
pipe = pipeline("summarization", model=model, tokenizer=tokenizer)  ## config=model_config, device_map="auto")

In [ ]:
# Summarize text into a job description.
@st.cache_data
def summarize(text: str) -> str:
    header = "Application to generate formal job descriptions using Salesforce/xgen LLM for summarization."
    text = header + "#### User: Please summarize this into a job description. \n\n" + text + "\n\n####"

    pipe_out = pipe(text)

    summary = pipe_out[0]["summary_text"]
    summary = "#### Job Description: ".join(pipe_out[0]["summary_text"]) 
                                            ##.split("#### Job Description: ")[:2]) + "\n\n####" + pipe_out[0]["summary_text"] + "\n\n####"  #tokenizer.decode(outputs[0], skip_special_tokens=True)  ##.lstrip()
    return summary

In [ ]:
summa = pipeline(task="summarization", model="Salesforce/xgen-7b-4k-base", revision="main")

In [ ]:
inputs = tokenizer("The world is", return_tensors="pt")
sample = model.generate(**inputs, max_length=128)
print(tokenizer.decode(sample[0]))

In [23]:
#Import Python Libraries
import pandas as pd
import datetime as dt
import folium
import geopandas as gpd
import streamlit as st
from streamlit_folium import folium_static

@st.cache_data
def read_csv(path):
    return pd.read_csv(path, compression='gzip', sep='\t', quotechar='"')

housing_price_df = read_csv('../input/state_market_tracker.tsv000.gz')
housing_price_df = housing_price_df[['period_begin','period_end','period_duration','property_type','median_sale_price','median_sale_price_yoy','homes_sold','state_code']]
housing_price_df = housing_price_df[(housing_price_df['period_begin']>='2022-07-01') & (housing_price_df['period_begin']<='2023-07-01')]

@st.cache_data
def read_file(path):
    return gpd.read_file(path)

#Read the geojson file
gdf = read_file('../input/georef-united-states-of-america-state.geojson')

#Merge the housing market data and geojson file into one dataframe
df_final = gdf.merge(housing_price_df, left_on="ste_stusps_code", right_on="state_code", how="outer").reset_index(drop=True)
df_final = df_final[['period_begin','period_end','period_duration','property_type','median_sale_price','median_sale_price_yoy',
                     'homes_sold','state_code','ste_stusps_code','geometry']] #'ste_code','ste_name','ste_area_code','ste_type','ste_stusps_code'
# df_final = df_final[~df_final['period_begin'].isna()].reset_index(drop=True)
# df_final = df_final[~df_final['median_sale_price'].isna()].reset_index(drop=True)
# df_final = df_final[~df_final['median_sale_price_yoy'].isna()].reset_index(drop=True)
# df_final = df_final[~df_final['homes_sold'].isna()].reset_index(drop=True)

# df_final = df_final.dropna(axis=0, how='any', inplace=True, subset= ['period_begin'])                     
                                                                    # ['period_begin','property_type', 'median_sale_price', 'median_sale_price_yoy',
                                                                    # 'homes_sold']) #.reset_index(drop=True) #, ignore_index=True)

####df = df_final.drop(['ste_code', 'ste_name', 'ste_area_code', 'ste_type', 'ste_stusps_code'], axis=1)

# df_final = df_final.dropna().reset_index(drop=True)


2023-09-03 20:09:29.302 No runtime found, using MemoryCacheStorageManager
2023-09-03 20:09:31.096 No runtime found, using MemoryCacheStorageManager


In [ ]:
# Instantiate model configuration
# model_config = AutoConfig.from_pretrained(Path(SHARDS))

# config = AutoConfig.from_pretrained(CONFIG)
# model = AutoModel.from_pretrained(SNAP)  ##[SHARD1, SHARD2, SHARD3]) #, config=config)
# # model = AutoModel.from_config(model_config)

# # # model.load_state_dict(torch.load(Path(config.SHARDS)))  ##, map_location="cpu"))
# # # print(model)

# tokenizer = AutoTokenizer.from_pretrained(SALESFORCE, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(SALESFORCE)

In [24]:
df_final.head()

,period_begin,period_end,period_duration,property_type,median_sale_price,median_sale_price_yoy,homes_sold,state_code,ste_stusps_code,geometry
0,2023-04-01,2023-04-30,30.0,All Residential,597700.0,-0.080654,6814.0,WA,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
1,2022-08-01,2022-08-31,30.0,Single Family Residential,613000.0,0.036288,8965.0,WA,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
2,2023-06-01,2023-06-30,30.0,All Residential,620500.0,-0.020862,8777.0,WA,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
3,2023-07-01,2023-07-31,30.0,Single Family Residential,633600.0,0.010260,6458.0,WA,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
4,2022-11-01,2022-11-30,30.0,Townhouse,629900.0,0.066748,324.0,WA,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."


In [25]:
df_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3133 entries, 0 to 3132
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   period_begin           3126 non-null   object  
 1   period_end             3126 non-null   object  
 2   period_duration        3126 non-null   float64 
 3   property_type          3126 non-null   object  
 4   median_sale_price      3126 non-null   float64 
 5   median_sale_price_yoy  3126 non-null   float64 
 6   homes_sold             3126 non-null   float64 
 7   state_code             3126 non-null   object  
 8   ste_stusps_code        3133 non-null   object  
 9   geometry               3133 non-null   geometry
dtypes: float64(4), geometry(1), object(5)
memory usage: 244.9+ KB


In [30]:
df_nan = df_final[df_final.isna()]

In [31]:
df_nan.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3133 entries, 0 to 3132
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   period_begin           0 non-null      object  
 1   period_end             0 non-null      object  
 2   period_duration        0 non-null      float64 
 3   property_type          0 non-null      object  
 4   median_sale_price      0 non-null      float64 
 5   median_sale_price_yoy  0 non-null      float64 
 6   homes_sold             0 non-null      float64 
 7   state_code             0 non-null      object  
 8   ste_stusps_code        0 non-null      object  
 9   geometry               0 non-null      geometry
dtypes: float64(4), geometry(1), object(5)
memory usage: 244.9+ KB


In [32]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   geo_point_2d     56 non-null     object  
 1   year             56 non-null     object  
 2   ste_code         56 non-null     object  
 3   ste_name         56 non-null     object  
 4   ste_area_code    56 non-null     object  
 5   ste_type         56 non-null     object  
 6   ste_stusps_code  56 non-null     object  
 7   ste_fp_code      0 non-null      float64 
 8   ste_gnis_code    56 non-null     object  
 9   geometry         56 non-null     geometry
dtypes: float64(1), geometry(1), object(8)
memory usage: 4.5+ KB


In [28]:
df_final.tail(10)

,period_begin,period_end,period_duration,property_type,median_sale_price,median_sale_price_yoy,homes_sold,state_code,ste_stusps_code,geometry
3123,2023-05-01,2023-05-31,30.0,Multi-Family (2-4 Unit),268700.0,-0.046864,43.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3124,2022-09-01,2022-09-30,30.0,Single Family Residential,240500.0,0.073511,4251.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3125,2023-07-01,2023-07-31,30.0,All Residential,260900.0,0.069655,3973.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3126,2023-03-01,2023-03-31,30.0,Townhouse,238000.0,-0.008424,103.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3127,2022-10-01,2022-10-31,30.0,Townhouse,232300.0,0.119889,88.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3128,2022-11-01,2022-11-30,30.0,Multi-Family (2-4 Unit),200700.0,-0.048134,36.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3129,2023-01-01,2023-01-31,30.0,Condo/Co-op,187300.0,0.090090,129.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3130,2022-09-01,2022-09-30,30.0,Condo/Co-op,199200.0,0.165166,217.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3131,2022-07-01,2022-07-31,30.0,Multi-Family (2-4 Unit),214800.0,0.011690,51.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."
3132,2022-11-01,2022-11-30,30.0,Single Family Residential,241900.0,0.065431,3100.0,KY,KY,"MULTIPOLYGON (((-89.48543 36.49749, -89.48266 ..."


In [26]:
df = df_final.copy()

In [27]:
df = df.dropna(axis=0, how='any', inplace=True) #, subset= ['period_begin']).reset_index(drop=True)
df = df.reset_index(drop=True)
df.info()

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [ ]:
df_final = df_final.fillna(0) #.reset_index(drop=True)

df_final = df_final.rename(columns={'period_begin':"Period",'property_type':"Type of Property",'median_sale_price':"Median Sale Price",
                                    'median_sale_price_yoy':"Median Sale Price YoY",'homes_sold':"Homes Sold",'state_code':"State",
                                    'geometry':"Location"})

df_final["Median Sale Price"] = df_final["Median Sale Price"].astype(int)
df_final["Median Sale Price YoY"] = df_final["Median Sale Price YoY"].astype(int)
df_final["Homes Sold"] = df_final["Homes Sold"].astype(int)
df_final["Month"] = pd.to_datetime(df_final["Period"], format='%Y-%m-%d').dt.to_period('M')

In [1]:
#Import Python Libraries
import pandas as pd
import folium
import geopandas as gpd
from folium.features import GeoJsonPopup, GeoJsonTooltip
import streamlit as st
from streamlit_folium import folium_static

#@st.cache
def read_csv(path):
    return pd.read_csv(path, compression='gzip', sep='\t', quotechar='"')

housing_price_df = read_csv('../input/state_market_tracker.tsv000.gz')
housing_price_df = housing_price_df[['period_begin','period_end','period_duration','property_type','median_sale_price','median_sale_price_yoy','homes_sold','state_code']]
housing_price_df = housing_price_df[(housing_price_df['period_begin']>='2022-01-01') & (housing_price_df['period_begin']<='2023-09-01')].reset_index(drop=True)

In [2]:
hp_df = housing_price_df.copy()

In [3]:
hp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4574 entries, 0 to 4573
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   period_begin           4574 non-null   object 
 1   period_end             4574 non-null   object 
 2   period_duration        4574 non-null   int64  
 3   property_type          4574 non-null   object 
 4   median_sale_price      4574 non-null   int64  
 5   median_sale_price_yoy  4574 non-null   float64
 6   homes_sold             4574 non-null   int64  
 7   state_code             4574 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 286.0+ KB


In [7]:
#@st.cache
def read_file(path):
    return gpd.read_file(path)

#Read the geojson file
gdf = read_file('../input/georef-united-states-of-america-state.geojson')

In [33]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   geo_point_2d     56 non-null     object  
 1   year             56 non-null     object  
 2   ste_code         56 non-null     object  
 3   ste_name         56 non-null     object  
 4   ste_area_code    56 non-null     object  
 5   ste_type         56 non-null     object  
 6   ste_stusps_code  56 non-null     object  
 7   ste_fp_code      0 non-null      float64 
 8   ste_gnis_code    56 non-null     object  
 9   geometry         56 non-null     geometry
dtypes: float64(1), geometry(1), object(8)
memory usage: 4.5+ KB


In [36]:
gdf.head(2)

,geo_point_2d,year,ste_code,ste_name,ste_area_code,ste_type,ste_stusps_code,ste_fp_code,ste_gnis_code,geometry
0,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
1,"{'lon': -106.10844349925604, 'lat': 34.4212989...",2022,[35],[New Mexico],USA,state,NM,NaN,00897535,"POLYGON ((-106.52805 31.78389, -106.52706 31.7..."


In [37]:
hp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4574 entries, 0 to 4573
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   period_begin           4574 non-null   object 
 1   period_end             4574 non-null   object 
 2   period_duration        4574 non-null   int64  
 3   property_type          4574 non-null   object 
 4   median_sale_price      4574 non-null   int64  
 5   median_sale_price_yoy  4574 non-null   float64
 6   homes_sold             4574 non-null   int64  
 7   state_code             4574 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 286.0+ KB


In [38]:
hp_df.head(64)

,period_begin,period_end,period_duration,property_type,median_sale_price,median_sale_price_yoy,homes_sold,state_code
0,2022-08-01,2022-08-31,30,Multi-Family (2-4 Unit),358300,0.111868,318,TX
1,2023-04-01,2023-04-30,30,All Residential,597700,-0.080654,6814,WA
2,2023-03-01,2023-03-31,30,Single Family Residential,501000,-0.040199,3276,OR
3,2022-11-01,2022-11-30,30,Single Family Residential,419100,0.097434,17596,FL
4,2022-02-01,2022-02-28,30,Condo/Co-op,244200,0.094606,347,PA
...,...,...,...,...,...,...,...,...
59,2023-05-01,2023-05-31,30,All Residential,372500,0.058123,1309,ME
60,2022-04-01,2022-04-30,30,Townhouse,296000,0.008518,3,RI
61,2023-06-01,2023-06-30,30,Single Family Residential,507800,0.032561,7323,NJ
62,2023-02-01,2023-02-28,30,Condo/Co-op,313800,-0.049445,83,RI


In [39]:
#Merge the housing market data and geojson file into one dataframe
df_final = gdf.merge(hp_df, left_on="ste_stusps_code", right_on="state_code", how="outer").reset_index(drop=True)

In [40]:
df_final.head(64)

,geo_point_2d,year,ste_code,ste_name,ste_area_code,ste_type,ste_stusps_code,ste_fp_code,ste_gnis_code,geometry,period_begin,period_end,period_duration,property_type,median_sale_price,median_sale_price_yoy,homes_sold,state_code
0,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2023-04-01,2023-04-30,30.0,All Residential,597700.0,-0.080654,6814.0,WA
1,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2022-02-01,2022-02-28,30.0,All Residential,579800.0,0.154859,7148.0,WA
2,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2022-08-01,2022-08-31,30.0,Single Family Residential,613000.0,0.036288,8965.0,WA
3,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2022-05-01,2022-05-31,30.0,Townhouse,687000.0,0.129715,687.0,WA
4,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2023-06-01,2023-06-30,30.0,All Residential,620500.0,-0.020862,8777.0,WA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2022-05-01,2022-05-31,30.0,All Residential,653600.0,0.141186,11878.0,WA
60,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2023-07-01,2023-07-31,30.0,Condo/Co-op,436600.0,0.018121,713.0,WA
61,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2023-01-01,2023-01-31,30.0,Single Family Residential,567800.0,0.027200,3487.0,WA
62,"{'lon': -120.5928034897229, 'lat': 47.41161380...",2022,[53],[Washington],USA,state,WA,NaN,01779804,"POLYGON ((-117.03235 48.99920, -117.13490 48.9...",2022-01-01,2022-01-31,30.0,Condo/Co-op,402200.0,0.175419,627.0,WA


In [9]:
#Merge the housing market data and geojson file into one dataframe
df_final = gdf.merge(hp_df, left_on="ste_stusps_code", right_on="state_code", how="outer").reset_index(drop=True)
df_final = df_final[['period_begin','period_end','period_duration','property_type','median_sale_price','median_sale_price_yoy','homes_sold',
                     'state_code', 'ste_code','ste_name', 'ste_area_code', 'ste_type', 'ste_stusps_code','geometry']]
df_final = df_final[~df_final['period_begin'].isna()].reset_index(drop=True)

In [10]:
df = df_final.copy()

In [ ]:
#Add sidebar to the app
st.sidebar.markdown("### Redfin Housing Data")
st.sidebar.markdown("This app is built using Streamlit to help visualize activity in the U.S. real estate market. All data from: https://www.redfin.com/news/data-center/")
st.sidebar.markdown("Developed by Robert Schell: https://github.com/schellrw")
#Add title and subtitle to the main interface of the app
st.title("U.S. Real Estate Activity Heatmap")
st.markdown("Where are the hottest housing markets in the U.S.? Select the housing market metrics you are interested in and your insights are just a couple clicks away. Hover over the map to view more details.")

#Create three columns/filters
col1, col2, col3 = st.columns(3)

with col1:
     period_list = df_final["period_begin"].unique().tolist()
     period_list.sort(reverse=True)
     year_month = st.selectbox("Snapshot Month", period_list, index=0)

with col2:
     prop_type = st.selectbox(
                "View by Property Type", ['All Residential', 'Single Family Residential', 'Townhouse','Condo/Co-op','Single Units Only','Multi-Family (2-4 Unit)'] , index=0)

with col3:
     metrics = st.selectbox("Select Housing Metrics", ["median_sale_price","median_sale_price_yoy", "homes_sold"], index=0)

#Update the data frame accordingly based on user input
df = df[df["period_begin"]==year_month]
df = df[df["property_type"]==prop_type]
df = df[['period_begin','period_end','period_duration','property_type',metrics,'state_code','ste_code','ste_name','ste_area_code','ste_type','ste_stusps_code','geometry']]

#st.write(df)

#Initiate a folium map
m = folium.Map(location=[40, -100], zoom_start=4,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

#Plot Choropleth map using folium
choropleth1 = folium.Choropleth(
    geo_data='./us-state-boundaries.geojson',       # Geojson file for the United States
    name='Choropleth Map of U.S. Housing Prices',
    data=df,                                        # df from the data preparation and user selection
    columns=['state_code', metrics],                # 'state code' and 'metrics' are the two columns in the dataframe that we use to grab the median sales price for each state and plot it in the choropleth map
    key_on='feature.properties.ste_stusps_code',    # key in the geojson file that we use to grab the geometries for each state in order to add the geographical boundary layers to the map
    fill_color='YlGn',
    nan_fill_color="White",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Housing Market Metrics',
    highlight=True,
    line_color='black').geojson.add_to(m)

#Add tooltips to the map
geojson1 = folium.features.GeoJson(
               data=df,
               name='United States Housing Prices',
               smooth_factor=2,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               tooltip=folium.features.GeoJsonTooltip(
                   fields=['period_begin',
                           'period_end',
                           'name',
                           metrics,],
                   aliases=["Period Begin:",
                            'Period End:',
                            'State:',
                            metrics+":"],
                   localize=True,
                   sticky=False,
                   labels=True,
                   style="""
                       background-color: #F0EFEF;
                       border: 2px solid black;
                       border-radius: 3px;
                       box-shadow: 3px;
                   """,
                   max_width=800,),
                    highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                   ).add_to(m)
#folium_static(m)

In [13]:
#Import Python Libraries
import pandas as pd
import folium
import geopandas as gpd
from folium.features import GeoJsonPopup, GeoJsonTooltip
import streamlit as st
from streamlit_folium import folium_static

@st.cache_data
def read_csv(path):
    return pd.read_csv(path, compression='gzip', sep='\t', quotechar='"')

housing_price_df = read_csv('../input/state_market_tracker.tsv000.gz')
housing_price_df = housing_price_df[['period_begin','period_end','period_duration','property_type','median_sale_price','median_sale_price_yoy','homes_sold','state_code']]
housing_price_df = housing_price_df[(housing_price_df['period_begin']>='2022-01-01') & (housing_price_df['period_begin']<='2023-07-01')]

@st.cache_data
def read_file(path):
    return gpd.read_file(path)

#Read the geojson file
# gdf = read_file('./input/georef-united-states-of-america-state.geojson')
gdf = read_file('../input/georef-united-states-of-america-state.geojson')

#Merge the housing market data and geojson file into one dataframe
df_final = gdf.merge(housing_price_df, left_on="ste_stusps_code", right_on="state_code", how="outer").reset_index(drop=True)
df_final = df_final[['period_begin','period_end','period_duration','property_type','median_sale_price','median_sale_price_yoy','homes_sold',
                     'state_code','ste_code','ste_name','ste_area_code','ste_type','ste_stusps_code','geometry']]
df_final = df_final[~df_final['period_begin'].isna()].reset_index(drop=True)

2023-09-03 14:20:31.062 No runtime found, using MemoryCacheStorageManager


2023-09-03 14:20:32.412 No runtime found, using MemoryCacheStorageManager


In [14]:
df_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4574 entries, 0 to 4573
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   period_begin           4574 non-null   object  
 1   period_end             4574 non-null   object  
 2   period_duration        4574 non-null   float64 
 3   property_type          4574 non-null   object  
 4   median_sale_price      4574 non-null   float64 
 5   median_sale_price_yoy  4574 non-null   float64 
 6   homes_sold             4574 non-null   float64 
 7   state_code             4574 non-null   object  
 8   ste_code               4574 non-null   object  
 9   ste_name               4574 non-null   object  
 10  ste_area_code          4574 non-null   object  
 11  ste_type               4574 non-null   object  
 12  ste_stusps_code        4574 non-null   object  
 13  geometry               4574 non-null   geometry
dtypes: float64(4), geometry(1), obje

In [18]:
df_final.head(2)

,period_begin,period_end,period_duration,property_type,median_sale_price,median_sale_price_yoy,homes_sold,state_code,ste_code,ste_name,ste_area_code,ste_type,ste_stusps_code,geometry
0,2023-04-01,2023-04-30,30.0,All Residential,597700.0,-0.080654,6814.0,WA,[53],[Washington],USA,state,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
1,2022-02-01,2022-02-28,30.0,All Residential,579800.0,0.154859,7148.0,WA,[53],[Washington],USA,state,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."


In [19]:
df_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4574 entries, 0 to 4573
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   period_begin           4574 non-null   object  
 1   period_end             4574 non-null   object  
 2   period_duration        4574 non-null   float64 
 3   property_type          4574 non-null   object  
 4   median_sale_price      4574 non-null   float64 
 5   median_sale_price_yoy  4574 non-null   float64 
 6   homes_sold             4574 non-null   float64 
 7   state_code             4574 non-null   object  
 8   ste_code               4574 non-null   object  
 9   ste_name               4574 non-null   object  
 10  ste_area_code          4574 non-null   object  
 11  ste_type               4574 non-null   object  
 12  ste_stusps_code        4574 non-null   object  
 13  geometry               4574 non-null   geometry
dtypes: float64(4), geometry(1), obje

In [21]:
df_final = df_final.dropna()

In [22]:
df_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4574 entries, 0 to 4573
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   period_begin           4574 non-null   object  
 1   period_end             4574 non-null   object  
 2   period_duration        4574 non-null   float64 
 3   property_type          4574 non-null   object  
 4   median_sale_price      4574 non-null   float64 
 5   median_sale_price_yoy  4574 non-null   float64 
 6   homes_sold             4574 non-null   float64 
 7   state_code             4574 non-null   object  
 8   ste_code               4574 non-null   object  
 9   ste_name               4574 non-null   object  
 10  ste_area_code          4574 non-null   object  
 11  ste_type               4574 non-null   object  
 12  ste_stusps_code        4574 non-null   object  
 13  geometry               4574 non-null   geometry
dtypes: float64(4), geometry(1), obje

In [16]:
df_slim = df_final.drop(['ste_code', 'ste_name', 'ste_area_code', 'ste_type', 'ste_stusps_code'], axis=1)
df_slim = df_slim.rename(columns={'property_type':"Type of Property",'median_sale_price':"Median Sale Price",'median_sale_price_yoy':"Median Sale Price YoY",
                            'homes_sold':"Homes Sold",'state_code':"State"})

In [17]:
df_slim.head(2)

,period_begin,period_end,period_duration,Type of Property,Median Sale Price,Median Sale Price YoY,Homes Sold,State,geometry
0,2023-04-01,2023-04-30,30.0,All Residential,597700.0,-0.080654,6814.0,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
1,2022-02-01,2022-02-28,30.0,All Residential,579800.0,0.154859,7148.0,WA,"POLYGON ((-117.03235 48.99920, -117.13490 48.9..."
